# Funktionierende KI-Modelle

## Hier kommen KI-Modelle rein, die bei mindestens einem Teammitglied funktionieren

In [ ]:
# imports
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import RobertaForQuestionAnswering, AutoTokenizer
import torch

c:\Users\Alex Felker\anaconda3\envs\ki-modelle\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Roberta-Base-Squad2 - Question Answering Model

In [4]:
# question answering - roberta-large-squad
model_name = 'deepset/roberta-large-squad2'

model = RobertaForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

context = """The Very Grateful Mole. Once there lived a mole who knew everything about trees, and was best friendswith a badger, who described everything about them, as the mole could not see. One day, the mole thought, "hold on, if badger can see, then why can't I?" He desperately wanted to see. So he went round to badger's house and said,"please could you help me? because I am tired of this darkness." The badger replied," I can help you. I happen to know a very special secret potion." So the next day, the very kind badger went out to find the ingredients to the special potion. When he returned, he had in his paws: bark from a pine tree, sunflowerseeds from the meadow, sap from a young oak, water from the stream and gold leaves from the magic tree in the heart of the woods. He placed them all into acarved wooden bowl and hung it above his fire too cook, stirring it gently every few minutes. When it was cooled, he poured it into the mole's ears, saying, "sleep, so that you will wake up and be able to see." The next day, Mole woke up, opened his eyes and thought he was in a magical dream because he had never seen anything except darkness. Until badger came round and told him, "you are not in a dream, you can see!" Mole said, "this is fabulous! I can finally see what you actually look like!" Badger helped him by describing what everything was, so that mole would understand what his eyes were showing him. Mole said to badger, "I want to go and see the tree that helped me to see." So they strolled through the calm woods until they found the tree with gold leaves. Mole ran up to it and hugged it, because he was overjoyed to meet this little tree that had made a big difference to his life. But, looking up, he saw that the surrounding large oaks needed pruning because they were blocking the sunlight from the tree, causing the leaves to shrivel up. "We'd better call for owl to help us". Owl swooped over and asked what he could do for them. Badger explained, "these oaks need your sharp beak to cut back their branches so that the sunlight reaches our gold leaf tree." Owl agreed to help, and flew up immediately to use his beak to snip off the longest branches. Sunshine poured down onto the little tree and the leaves suddenly began to glow in response. From then on, badger and mole visited the tree everyday to make sure it had enough water and light. And all was peaceful in the wonderful woods."""

questions = [
    # 'Why did the large oaks needed pruning?',
    # 'Why did the Mole think he was in a magical dream?',
    # 'What ingridients did the badger get from the magic tree for the special secret potion?',
    "What did the badger pour into the mole's ears?"
]

QA_results = []

for question in questions:
    inputs = tokenizer(
        question,
        context,
        return_tensors='pt',
        max_length=512,
        truncation=True
    )

    with torch.no_grad():
        # outputs = model(**inputs)
        outputs = model(**inputs, return_dict=True)

    # Find start and end indices with highest logits
    start_idx = torch.argmax(outputs['start_logits']).item()
    end_idx = torch.argmax(outputs['end_logits']).item() + 1

    answer = tokenizer.decode(
        inputs['input_ids'][0][start_idx:end_idx], skip_special_tokens=True)

    QA_results.append({
        'question': question,
        'answer': answer
    })

for result in QA_results:
    display(f'Question: {result["question"]}')
    display(f'Answer  :{result["answer"]}')

"Question: What did the badger pour into the mole's ears?"

'Answer  : potion'

### Flan-T5 - Text to text Generation Model

In [8]:
# model_name = "google/flan-t5-xxl"     # hat 11B parameters, funkt bei meinem laptop nicht - Alex
model_name = "google/flan-t5-large"   # hat 783M parameters, funkt bei meinem laptop - Alex
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# alt + z um text zu wrapen
input_text = """Generate three questions about the following text: The Very Grateful Mole. Once there lived a mole who knew everything about trees, and was best friendswith a badger, who described everything about them, as the mole could not see. One day, the mole thought, "hold on, if badger can see, then why can't I?" He desperately wanted to see. So he went round to badger's house and said,"pleasecould you help me? because I am tired of this darkness." The badger replied," I can help you. I happen to know a very special secret potion." So the next day, the very kind badger went out to find the ingredients to the special potion. When he returned, he had in his paws: bark from a pine tree, sunflowerseeds from the meadow, sap from a young oak, water from the stream and gold leaves from the magic tree in the heart of the woods. He placed them all into acarved wooden bowl and hung it above his fire too cook, stirring it gently every few minutes. When it was cooled, he poured it into the mole's ears, saying, "sleep, so that you will wake up and be able to see." The next day, Mole woke up, opened his eyes and thought he was in a magical dream because he had never seen anything except darkness. Until badger came round and told him, "you are not in a dream, you can see!" Mole said, "this is fabulous! I can finally see what you actually look like!" Badger helped him by describing what everything was, so that mole would understand what his eyes were showing him. Mole said to badger, "I want to go and see the tree that helped me to see." So they strolled through the calm woods until they found the tree with gold leaves. Mole ran up to it and hugged it, because he was overjoyed to meet this little tree that had made a big difference to his life. But, looking up, he saw that the surrounding large oaks needed pruning because they were blocking the sunlight from the tree, causing the leaves to shrivel up. "We'd better call for owl to help us". Owl swooped over and asked what he could do for them. Badger explained, "these oaks need your sharp beak to cut back their branches so that the sunlight reaches our gold leaf tree." Owl agreed to help, and flew up immediately to use his beak to snip off the longest branches. Sunshine poured down onto the little tree and the leaves suddenly began to glow in response. From then on, badger and mole visited the tree everyday to make sure it had enough water and light. And all was peaceful in the wonderful woods."""
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(
    input_ids,
    max_length=512,
    num_beams=4,
    early_stopping=True,
    no_repeat_ngram_size=5,
)

decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
print(decoded_output)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Token indices sequence length is longer than the specified maximum sequence length for this model (628 > 512). Running this sequence through the model will result in indexing errors


What did the badger pour into the mole's ears?
